In [203]:
import pandas as pd

In [204]:
data = pd.read_excel("Courses.xlsx", engine="openpyxl")

In [205]:
data.columns

Index(['Unnamed: 0', 'Year', 'Period', 'Code', 'Name', 'Section', 'Dept.', 'T',
       'P', 'L', 'Cr', 'ECTS', 'Category', 'Lecturer', 'Room', 'Schedule',
       '# of Students', 'Successfull', 'Unsuccessfull', 'Conditional',
       'Withdrawn', 'Average', 'Staff ID'],
      dtype='object')

In [206]:
"""
data.Schedule
data.Section
data.Code
"""

'\ndata.Schedule\ndata.Section\ndata.Code\n'

In [207]:
drop_columns = ['Unnamed: 0', 'Year', 'Period', 'Name', 'Dept.', 'T',
       'P', 'L', 'Cr', 'ECTS', 'Category', 'Lecturer', 'Room',
       '# of Students', 'Successfull', 'Unsuccessfull', 'Conditional',
       'Withdrawn', 'Average', 'Staff ID']
data.drop(drop_columns, axis=1, inplace=True)


In [208]:
data[80:100]

,Code,Section,Schedule
80,CITY 111,CITY 111_01,Th 09 - 13
81,CITY 111-O,CITY 111-O_1,Th 09 - 13
82,CITY 121,CITY 121_01,We 14 - 17
83,CITY 201-O,CITY 201-O_1,Tu/Fr 09 - 12 Tu/Fr 13 - 15
84,CITY 211,CITY 211_01,Mo 09 - 12
85,CITY 211-O,CITY 211-O_1,Mo 09 - 12
86,CITY 213,CITY 213_01,Th 09 - 12
87,CITY 213-O,CITY 213-O_1,Th 09 - 12
88,CITY 301,CITY 301_01,Tu/Fr 13 - 15 Tu/Fr 09 - 12
89,CITY 399,CITY 399_01,Fr 18 - 19


In [223]:
from collections import defaultdict
import re

class Section:
    def __init__(self, section_info):
        self.days = []
        self.start_times = []
        self.end_times = []

        for info in section_info:
            self.days.append(info[0])
            self.start_times.append(info[1])
            self.end_times.append(info[2])

    def __str__(self):
        return f"Days: {self.days}, Start Times: {self.start_times}, End Times: {self.end_times}"
    
class Scheduler:
    def __init__(self):
        self.courses = {}
        self.schedule = defaultdict(list)

    def extract_days_and_times(self, section):
        """
        Extracts days and times from the section string.
        """
        pattern = r'(\w{2})\s(\d{2})\s-\s(\d{2})'
        matches = re.findall(pattern, section)
        
        results = []
        for match in matches:
            day, start_time, end_time = match
            results.append((day, int(start_time), int(end_time)))
        
        return results

    def add_course(self, code, data):
        """
        Adds a course with its sections.
        """
        sections = [Section(self.extract_days_and_times(x)) for x in data[data['Code'] == code]['Schedule'].tolist()]
        self.courses[code] = {"code": code, "sections": sections}

    def print_courses(self):
        """
        Prints the details of all courses and their sections.
        """
        if not self.courses:
            print("No courses available.")
            return
        
        for code, course_info in self.courses.items():
            print(f"Course Code: {code}")
            for section in course_info["sections"]:
                print(f"   {section}")

sc = Scheduler()
sc.add_course("CITY 430", data)
sc.add_course("CMPE 112", data)
sc.add_course("CMPE 113", data)
sc.print_courses()

Course Code: CITY 430
   Days: ['Th'], Start Times: [13], End Times: [16]
Course Code: CMPE 112
   Days: ['We', 'Th'], Start Times: [14, 12], End Times: [16, 14]
   Days: ['Tu', 'We'], Start Times: [9, 16], End Times: [11, 18]
   Days: ['Mo', 'Fr'], Start Times: [9, 9], End Times: [11, 11]
Course Code: CMPE 113
   Days: ['We', 'Th'], Start Times: [14, 12], End Times: [16, 14]
   Days: ['Tu', 'We'], Start Times: [9, 16], End Times: [11, 18]
   Days: ['Mo', 'Fr'], Start Times: [9, 9], End Times: [11, 11]


In [235]:
from itertools import product
from collections import defaultdict
import re
import pandas as pd

class Section:
    def __init__(self, section_info):
        self.days = []
        self.start_times = []
        self.end_times = []

        for info in section_info:
            self.days.append(info[0])
            self.start_times.append(info[1])
            self.end_times.append(info[2])

    def __str__(self):
        return f"Days: {self.days}, Start Times: {self.start_times}, End Times: {self.end_times}"

class Scheduler:
    def __init__(self):
        self.courses = {}
        self.schedule = defaultdict(list)

    def extract_days_and_times(self, section):
        """
        Extracts days and times from the section string.
        """
        pattern = r'(\w{2})\s(\d{2})\s-\s(\d{2})'
        matches = re.findall(pattern, section)
        
        results = []
        for match in matches:
            day, start_time, end_time = match
            results.append((day, int(start_time), int(end_time)))
        
        return results

    def add_course(self, code, data):
        """
        Adds a course with its sections.
        """
        sections = [Section(self.extract_days_and_times(x)) for x in data[data['Code'] == code]['Schedule'].tolist()]
        self.courses[code] = {"code": code, "sections": sections}

    def print_courses(self):
        """
        Prints the details of all courses and their sections.
        """
        if not self.courses:
            print("No courses available.")
            return
        
        for code, course_info in self.courses.items():
            print(f"Course Code: {code}")
            for section in course_info["sections"]:
                print(f"   {section}")

    def sections_conflict(self, sec1, sec2):
        """
        Checks if two sections conflict in time and day.
        """
        days1 = set(sec1.days)
        days2 = set(sec2.days)

        common_days = days1.intersection(days2)

        for day in common_days:
            for start1, end1 in zip(sec1.start_times, sec1.end_times):
                for start2, end2 in zip(sec2.start_times, sec2.end_times):
                    if not (end1 <= start2 or start1 >= end2):
                        return True
        return False

    def find_all_non_conflicting_schedules(self):
        """
        Finds all non-conflicting schedules by selecting one section per course.
        """
        course_codes = list(self.courses.keys())
        sections_lists = [self.courses[code]['sections'] for code in course_codes]
        
        all_combinations = product(*sections_lists)

        def has_conflict(combination):
            """
            Checks if the given combination of sections has any conflicts.
            """
            for i in range(len(combination)):
                for j in range(i + 1, len(combination)):
                    if self.sections_conflict(combination[i], combination[j]):
                        return True
            return False

        valid_schedules = []
        for combination in all_combinations:
            if not has_conflict(combination):
                valid_schedules.append(combination)

        if valid_schedules:
            print("Non-conflicting schedules found:")
            for idx, schedule in enumerate(valid_schedules):
                print(f"Schedule {idx + 1}:")
                for section in schedule:
                    print(section)
                print()
        else:
            print("No non-conflicting schedules found.")


sc = Scheduler()
sc.add_course("CMPE 327", data)
sc.add_course("CMPE 361", data)
sc.add_course("CMPE 371", data)
sc.add_course("SPA 101", data)
sc.add_course("ECON 102", data)
sc.add_course("PSY 104", data)
sc.add_course("CMPE 421", data)

sc.find_all_non_conflicting_schedules()


Non-conflicting schedules found:
Schedule 1:
Days: ['Mo'], Start Times: [13], End Times: [16]
Days: ['We', 'Fr'], Start Times: [9, 11], End Times: [11, 13]
Days: ['We'], Start Times: [13], End Times: [16]
Days: ['Th'], Start Times: [13], End Times: [16]
Days: ['Mo', 'Th'], Start Times: [9, 17], End Times: [12, 19]
Days: ['Tu'], Start Times: [15], End Times: [18]
Days: ['We'], Start Times: [18], End Times: [21]

Schedule 2:
Days: ['Mo'], Start Times: [13], End Times: [16]
Days: ['We', 'Fr'], Start Times: [9, 11], End Times: [11, 13]
Days: ['Fr'], Start Times: [14], End Times: [17]
Days: ['Th'], Start Times: [13], End Times: [16]
Days: ['Mo', 'Th'], Start Times: [9, 17], End Times: [12, 19]
Days: ['Tu'], Start Times: [15], End Times: [18]
Days: ['We'], Start Times: [18], End Times: [21]

